In [67]:
import sys
sys.path.append('/Users/tschip/workspace/baa/baa-ruefer/')
import json
from data_loaders.JSONDataLoader import JSONDataLoader

team_ids = [487, 489]
dataloader = JSONDataLoader(team_ids)
player_names = dataloader.get_player_names()
player_information = dataloader.get_player_information()
player_statistics = dataloader.get_player_statistics()
player_injuries = dataloader.get_player_injuries()
team_information = dataloader.get_team_information()
venue_information = dataloader.get_venue_information()
team_statistics = dataloader.get_team_statistics()
player_transfers = dataloader.get_player_transfers()
player_news = dataloader.get_player_news()
news_details = dataloader.get_news_details()
team_news = dataloader.get_team_news()
head_to_head = dataloader.get_head_to_head()
home_team = dataloader.get_home_team_id()
players_ids = dataloader.get_players_ids()
team_fixtures = dataloader.get_team_fixtures()
todays_fixture_id = dataloader.get_todays_fixture_id()
fixture_lineup = dataloader.get_fixture_lineup()
fixture_statistics = dataloader.get_fixture_stats()
coaches = dataloader.get_coaches()

team_information_parentT = []
team_statistics_parentT = []
team_injuries_parentT = []
venue_information_parentT = []
player_information_parentT = []
player_statistics_parentT = []
player_transfers_parentT = []
fixture_information_parentT = []
fixture_statistics_parentT = []
coach_parentT = []

with open('llama_output_old.json', 'r') as f:
    data = json.load(f)

In [68]:
def get_fixture_information():
    fixtures = team_fixtures[list(team_fixtures.keys())[0]]
    for fixture in fixtures:
        if fixture == todays_fixture_id:
            fixture =  team_fixtures[str(home_team)][fixture]
            try:
                del fixture['fixture_id']
                del fixture['league_id']
                del fixture['league_logo']
                del fixture['league_name']
                del fixture['league_country']
                del fixture['league_flag']
                del fixture['league_season']
                del fixture['teams_home_id']
                del fixture['teams_away_id']
            except:
                pass

            return fixture

In [69]:
import numpy as np
from nltk import ngrams
from difflib import SequenceMatcher
import re

def word_overlap_model(g, table_lexical_items):
        return sum(1 for token in g if token in table_lexical_items) / len(g)

def entailed_precision(generated_ngrams, table_lexical_items):
    total_entailment_prob = 0
    total_generated_ngrams = 0
    for g in generated_ngrams:
        total_entailment_prob += word_overlap_model(g, table_lexical_items)
        total_generated_ngrams += len(g)
    return total_entailment_prob / total_generated_ngrams if total_generated_ngrams > 0 else 0

def geometric_average(scores):
    return np.exp(np.mean(np.log([score + 1e-10 for score in scores]))) if len(scores) > 0 else 0

def longest_common_subsequence(x, y):
    x = str(x)
    y = str(y)
    seq_matcher = SequenceMatcher(None, x, y)
    match = seq_matcher.find_longest_match(0, len(x), 0, len(y))
    return match.size

def entailment_recall(table_records, generated_text):
    total_recall = sum(longest_common_subsequence(record, generated_text) for record in table_records)
    return total_recall / len(table_records) if len(table_records) > 0 else 0

def parent_t_score(generated_text, table_records):
    precision_scores = [entailed_precision(ngrams(generated_text.split(), n), set(table_records)) for n in range(1, 6)]
    entailed_precision_score = geometric_average(precision_scores)
    recall_score = entailment_recall(table_records, generated_text)
    parent_t = (2 * entailed_precision_score * recall_score) / (entailed_precision_score + recall_score) if (entailed_precision_score + recall_score) > 0 else 0
    return parent_t

def system_level_parent_t_score(model_outputs, table_records_list):
        total_parent_t_score = sum(parent_t_score(generated_text, table_records) for generated_text, table_records in zip(model_outputs, table_records_list))
        return total_parent_t_score / len(model_outputs) if len(model_outputs) > 0 else 0

def correct_json(json_string):
        # Remove extra spaces and newlines
        cleaned_json = json_string.replace("\n", "").replace("  ", "")
        
        # Remove double quotes around property names
        cleaned_json = cleaned_json.replace('"{', '{').replace('}"', '}')
        cleaned_json = cleaned_json.replace(',}', '}')

        return cleaned_json

def extract_first_match(input_string):
        pattern = r'\{[^}]*\}'
        matches = re.findall(pattern, input_string)
        
        if matches:
            return matches[0]
        else:
            return input_string

In [70]:
for team_id in team_ids:
    li = []
    for item in team_information[team_id].items():
        for i in str(item[1]).split(' '):
            li.append(i)

    team_information_parentT.append(parent_t_score(data[str(team_id)]['information'], li))

In [71]:
[entailed_precision(ngrams(data[str(team_id)]['information'].split(), n), set(li)) for n in range(1, 6)]

[0.041666666666666664,
 0.010869565217391304,
 0.00505050505050505,
 0.002976190476190476,
 0.002]

In [72]:
sum(team_information_parentT) / len(team_information_parentT)

0.006698791044255771

In [73]:
for team_id in team_ids:
    team_statistics_output = data[str(team_id)]['statistics']
    try:
        model_outputs = json.loads(correct_json(extract_first_match(team_statistics_output)))
        output_strings = []
        for statistic in model_outputs:
            output_strings.append(model_outputs[statistic])

        li = []
        for item in team_statistics(team_id).items():
            for i in str(item[1]).split(' '):
                li.append(i)

        team_statistics_parentT.append(system_level_parent_t_score(output_strings, li))

    except:
        team_statistics_parentT.append(0)

In [74]:
player_transfers

{'1835': [{'date': '2022-07-14',
   'teams_in_name': 'Brentford',
   'teams_out_name': 'Lazio',
   'transfer_type': 'Free'},
  {'date': '2016-07-01',
   'teams_in_name': 'Lazio',
   'teams_out_name': 'Salernitana',
   'transfer_type': 'N/A'},
  {'date': '2015-07-17',
   'teams_in_name': 'Salernitana',
   'teams_out_name': 'Lazio',
   'transfer_type': 'Loan'}],
 '1846': [{'date': '2008-01-01',
   'teams_in_name': 'Lazio',
   'teams_out_name': 'Dinamo Bucuresti',
   'transfer_type': '€ 1M'}],
 '1836': [{'date': '2023-07-07',
   'teams_in_name': 'Inter',
   'teams_out_name': 'Lazio',
   'transfer_type': '€ 4M'},
  {'date': '2022-09-01',
   'teams_in_name': 'Inter',
   'teams_out_name': 'Lazio',
   'transfer_type': 'Loan'},
  {'date': '2018-07-11',
   'teams_in_name': 'Lazio',
   'teams_out_name': 'Sassuolo',
   'transfer_type': '€ 10M'},
  {'date': '2013-07-08',
   'teams_in_name': 'Sassuolo',
   'teams_out_name': 'Genoa',
   'transfer_type': '€ 1.7M'},
  {'date': '2013-07-01',
   'teams_

In [75]:
for team_id in team_ids:
    information = player_information[str(team_id)]
    statistics = player_statistics[str(team_id)]
    transfers = player_transfers

    for player_id in data[str(team_id)]['players'].keys():
        try:
            model_outputs = data[str(team_id)]['players'][player_id]['information']
            output_string = ''
            for item in model_outputs:
                output_string += model_outputs[item]

            li = []
            for item in information[int(player_id)].items():
                for i in str(item[1]).split(' '):
                    li.append(i)

            player_information_parentT.append(parent_t_score(output_string, li))
        except:
            player_information_parentT.append(0)

        player_statistics_output = data[str(team_id)]['players'][player_id]['statistics']
        try:
            model_outputs = json.loads(correct_json(extract_first_match(player_statistics_output)))
            output_strings = []
            for statistic in model_outputs:
                output_strings.append(model_outputs[statistic])

            li = []
            for item in statistics[player_id].items():
                for i in str(item[1]).split(' '):
                    li.append(i)

            player_statistics_parentT.append(system_level_parent_t_score(output_strings, li))
        except Exception as e:
            player_statistics_parentT.append(0)

        player_transfers = data[str(team_id)]['players'][player_id]['transfers']
        try:
            model_outputs = json.loads(correct_json(extract_first_match(player_transfers)))
            output_strings = []
            for transfer in model_outputs:
                output_strings.append(model_outputs[transfer])

            li = []
            for item in transfers[player_id].items():
                for i in str(item[1]).split(' '):
                    li.append(i)

            player_transfers_parentT.append(system_level_parent_t_score(output_strings, li))
        except Exception as e:
            player_transfers_parentT.append(0)
        

In [76]:
for team_id in team_ids:
    team_coach_output = data[str(team_id)]['coach']
    try:
        model_outputs = json.loads(correct_json(extract_first_match(team_coach_output)))
        output_strings = []
        for statistic in model_outputs:
            output_strings.append(model_outputs[statistic])

        li = []
        for item in coaches[str(team_id)].items():
            for i in str(item[1]).split(' '):
                li.append(i)

        coach_parentT.append(system_level_parent_t_score(output_strings, li))
    except Exception as e:
        coach_parentT.append(0)

In [77]:
information = get_fixture_information()
fixture_output = data['fixture']['information']
li = []
for item in information.items():
    for i in str(item[1]).split(' '):
        li.append(i)

fixture_information_parentT.append(parent_t_score(fixture_output, li))

for team_id in team_ids:
    fixture_statistics_output = data['fixture'][str(team_id)]['statistics']
    try:
        model_outputs = json.loads(correct_json(extract_first_match(fixture_statistics_output)))
        output_strings = []
        for statistic in model_outputs:
            output_strings.append(model_outputs[statistic])

        li = []
        for item in fixture_statistics[str(team_id)].items():
            for i in str(item[1]).split(' '):
                li.append(i)

        fixture_statistics_parentT.append(system_level_parent_t_score(output_strings, li))
    except Exception as e:
        fixture_statistics_parentT.append(0)
    

In [78]:
model_outputs = data['venue']
li = []
for item in venue_information.items():
    for i in str(item[1]).split(' '):
        li.append(i)

venue_information_parentT.append(parent_t_score(model_outputs, li))

In [81]:
print(f'Team information: {sum(team_information_parentT) / len(team_information_parentT)}')
print(f'eam statistics: {sum(team_statistics_parentT) / len(team_statistics_parentT)}')
print(f'venue info{sum(venue_information_parentT) / len(venue_information_parentT)}')
print(f'player info{sum(player_information_parentT) / len(player_information_parentT)}')
print(f'palyer stats {sum(player_statistics_parentT) / len(player_statistics_parentT)}')
print(f'player transfer {sum(player_transfers_parentT) / len(player_transfers_parentT)}')
print(f'fixture info{sum(fixture_information_parentT) / len(fixture_information_parentT)}')
print(f'ficture stats {sum(fixture_statistics_parentT) / len(fixture_statistics_parentT)}')
print(f'coach {sum(coach_parentT) / len(coach_parentT)}')

Team information: 0.006698791044255771
eam statistics: 0.0
venue info0.01443490236167194
player info0.10182876070332085
palyer stats 0.0
player transfer 0.0
fixture info0.12959428572773599
ficture stats 0.0
coach 0.0
